## Łańcuch Markowa (Markov chain)

Łańcuchy Markowa modelują procesy stochastyczne, które ewoluują w czasie zgodnie z wewnętrzną dynamiką.
**Stan** jest to kompletny zbiór informacji, który opisuje sytuację, w jakiej znalazło się dane środowisko. Jeżeli *tranzycja* do kolejnego stanu, zależy jedynie od stanu aktualnego (nie rozpatrujemy pod uwagę tych stanów, w których środowisko już było wcześniej), mówimy, że proces posiada **własność Markowa**.
![](img/markov_property.png)
![](img/robot_markov_chain.png)
Powyższy obrazek przedstawia zepsutego robota, który losowo porusza się po środowisku. Robot porusza się w zadanym kierunku zgodnie z prawdopodobieństwem określonym na rysunku. Będąc w stanie (1,2), nie ma znaczenia, jaką drogę przebył, aby się tam znaleźć. Przejście do kolejnego stanu zależy jedynie od aktualnego (własność Markowa).
![](img/absorbing.png)
Załóżmy, że jeżeli robot uderzy w ścianę (tj. poruszy się w jej kierunku) rozbija się i nie może się już poruszyć. Taki stan nazywamy *terminalnym*, który kończy *epizod*.

## Proces nagród Markowa (Markov reward process)

Każdy system posiada stany, w których lepiej byłoby się znajdować i takie, w których mniej. Łańcuch Markowa nie mówi nam nic na temat przewagi jednych stanów nad drugimi. Do tego potrzebujemy **nagród** przypisanych do poszczególnych stanów procesu.

### Przykład

Rozpatrzmy przykładowe środowisko typu *grid world* o wielkości 3x3 (stany indeksowane są następująco: (0,0):1, (0,1):2, ... , (2,2):9) ze stanem terminalnym, do którego agent przechodzi, gdy uderzy w ścianę.

In [1]:
import numpy as np

In [2]:
m = 3
m2 = m ** 2  # liczba stanów
q = np.zeros(m2)
q[m2 // 2] = 1

In [3]:
q

array([0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [4]:
ix_map = {i + 1: (i // m, i % m) for i in range(m2)}
ix_map

{1: (0, 0),
 2: (0, 1),
 3: (0, 2),
 4: (1, 0),
 5: (1, 1),
 6: (1, 2),
 7: (2, 0),
 8: (2, 1),
 9: (2, 2)}

In [5]:
# funkcja tworząca macierz przejść z określonymi prawdopodobieństwami tranzycji pomiędzy stanami
def get_P(m, p_up, p_down, p_left, p_right):
    m2 = m ** 2
    P = np.zeros((m2, m2))
    ix_map = {i + 1: (i // m, i % m) for i in range(m2)}
    for i in range(m2):
        for j in range(m2):
            r1, c1 = ix_map[i + 1]
            r2, c2 = ix_map[j + 1]
            rdiff = r1 - r2
            cdiff = c1 - c2
            if rdiff == 0:
                if cdiff == 1:
                    P[i, j] = p_left
                elif cdiff == -1:
                    P[i, j] = p_right
                elif cdiff == 0:
                    if r1 == 0:
                        P[i, j] += p_down
                    elif r1 == m - 1:
                        P[i, j] += p_up
                    if c1 == 0:
                        P[i, j] += p_left
                    elif c1 == m - 1:
                        P[i, j] += p_right
            elif rdiff == 1:
                if cdiff == 0:
                    P[i, j] = p_down
            elif rdiff == -1:
                if cdiff == 0:
                    P[i, j] = p_up
    return P

# Poszerzenie macierzy P o stan terminalny 'crashed'
P = np.zeros((m2 + 1, m2 + 1))
P[:m2, :m2] = get_P(3, 0.2, 0.3, 0.25, 0.25)
for i in range(m2):
    P[i, m2] = P[i, i]
    P[i, i] = 0
P[m2, m2] = 1

In [6]:
P2 = get_P(3, 0.2, 0.3, 0.25, 0.25)
P2

array([[0.55, 0.25, 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.25, 0.3 , 0.25, 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.25, 0.55, 0.  , 0.  , 0.2 , 0.  , 0.  , 0.  ],
       [0.3 , 0.  , 0.  , 0.25, 0.25, 0.  , 0.2 , 0.  , 0.  ],
       [0.  , 0.3 , 0.  , 0.25, 0.  , 0.25, 0.  , 0.2 , 0.  ],
       [0.  , 0.  , 0.3 , 0.  , 0.25, 0.25, 0.  , 0.  , 0.2 ],
       [0.  , 0.  , 0.  , 0.3 , 0.  , 0.  , 0.45, 0.25, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.3 , 0.  , 0.25, 0.2 , 0.25],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.3 , 0.  , 0.25, 0.45]])

In [7]:
# końcowa macierz tranzycji, ze stanu 9 agent nie może już przejść do żadnego innego poza stanem 9.
# Ze stanu pierwszego istnieje 0.55 szansy, że agent uderzy w ścianę, 0.25, że przejdzie w prawo (stan 2) oraz
# 0.2 szansy że pójdzie do góry (stan 4)
P

array([[0.  , 0.25, 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.55],
       [0.25, 0.  , 0.25, 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.3 ],
       [0.  , 0.25, 0.  , 0.  , 0.  , 0.2 , 0.  , 0.  , 0.  , 0.55],
       [0.3 , 0.  , 0.  , 0.  , 0.25, 0.  , 0.2 , 0.  , 0.  , 0.25],
       [0.  , 0.3 , 0.  , 0.25, 0.  , 0.25, 0.  , 0.2 , 0.  , 0.  ],
       [0.  , 0.  , 0.3 , 0.  , 0.25, 0.  , 0.  , 0.  , 0.2 , 0.25],
       [0.  , 0.  , 0.  , 0.3 , 0.  , 0.  , 0.  , 0.25, 0.  , 0.45],
       [0.  , 0.  , 0.  , 0.  , 0.3 , 0.  , 0.25, 0.  , 0.25, 0.2 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.3 , 0.  , 0.25, 0.  , 0.45],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

Agent otrzymuje nagrodę 1 za każdy ruch. Gdy wpadnie w ścianę, epizod się kończy, a agent otrzymuje nagrodę równą 0.
Agent rozpoczyna epizod w stanie początkowym, a następne zdobyte przez agenta nagrody są zapisywane.
Pozwoli to później na określenie średniej (oczekiwanej) wartości **sumy nagród** (*return*, suma nagród w danym epizodzie) dla każdego ze stanów środowiska.
Taką wartość nazywamy **wartością stanu (state value)**.

In [8]:
n = 10 ** 5
avg_rewards = np.zeros(m2)
for s in range(9): # dla każdego stanu początkowego
    for i in range(n): # symulacja n epizodów
        crashed = False
        s_next = s
        episode_reward = 0
        while not crashed: # symulacja przejść, dopóki agent nie wpadnie w ścianę
            s_next = np.random.choice(m2 + 1, p=P[s_next, :])
            if s_next < m2: # wewnątrz ścian
                episode_reward += 1
            else: # zderzenie ze ścianą
                crashed = True
        avg_rewards[s] += episode_reward
avg_rewards /= n

In [9]:
np.round(avg_rewards, 2) # średnie sumy nagród

array([1.47, 2.1 , 1.48, 2.43, 3.43, 2.45, 1.99, 2.82, 2.  ])

### Zadanie #1

Który stan jest najlepszy, a który najgorszy i dlaczego? Z czego wynika przewaga jednego nad drugim?


Najlepszy stan: 5. Największa suma nagród (3.41). Najczęściej odwiedzany przez agenta. <br>
Najgorszy stan: 1 lub 3. Najmniejsza suma nagród (1.47 lub 1.48). Najrzadziej odwiedzane przez agenta. <br>

Dodatkowo stan 5 ma 4 sąsiadów, co oznacza, że szansa na odwiedzenie tego stanu przez agenta jest największa oraz w tym stanie agent nie może uderzyć w ścianę (p=0).

### Zadanie #2

Wartość stanu definiowana jest jako oczekiwana zdyskontowana suma nagród, gdy zaczynamy ze stanu *s*:
![](img/sv_mrp.png)
Co oznacza **współczynnik dyskontowania γ**? Jak go możemy interpretować?

Współczynnik dyskontowania reguluje względną ważność krótko i długoterminowych nagród. <br>
$$
{\gamma \in [0,1]\,}
$$
gdzie gdy jest blisko 0, to oznacza, że bierzemy pod uwagę krótkoterminowe nagrody, a jak blisko 1, to długoterminowe.

### Rekurencyjna relacja pomiędzy wartościami stanów

Możemy wyznaczyć wartość stanu ze stanów sąsiadujących (takich, do których agent może przejść ze stanu rozpatrywanego) zgodnie z formułą rekurencyjną:
![](img/recursive_relationship_formula.png)

In [10]:
# estymacja wartości (state value) czwartego stanu z wartości stanów sąsiadujących
(1 + 2.45) * 0.25 + (1 + 2.44) * 0.25 + 0.2 * (1+2.81) + 0.3*(1+2.12)

3.4205

Powyższą formułę nazywamy **równaniem Bellmana dla MRP** i możemy ją zapisać następująco (uwzględniając już wsp. dyskontowania):
![](img/bellman_eq_mrp.png)

### Zadanie #3

Wyznacz używając reguły rekurencyjnej wartości stanów dla pozostałych stanów środowiska (v(0,0), v(1,0), ..., v(2,2))

In [11]:
states = [(0,0), (1,0), (2,0), (0,1), (1,1), (2,1), (0,2), (1,2), (2,2)]
a = len(states)      
v = np.round(avg_rewards, 2)
for i in range(a):
    value = 0
    for j in range(a):
        value = value + P[i][j]*(1+v[j])
    print("v(" + str(states[i][0]) + "," + str(states[i][1]) + ") = " + str(value));

v(0,0) = 1.461
v(1,0) = 2.1235
v(2,0) = 1.465
v(0,1) = 2.4465
v(1,1) = 3.4140000000000006
v(2,1) = 2.4515000000000002
v(0,2) = 1.984
v(1,2) = 2.8265000000000002
v(2,2) = 1.9899999999999998


### Iteracyjna estymacja wartości stanów

Jedną z centralnych idei UzW jest iteracyjna możliwość estymacji wartości stanów.
Użyjmy równania Bellmana jako reguły aktualizacji wartości.
Estymację kończymy, gdy aktualizacja dla wartości stanu jest mniejsza niż zadana wartość progowa (threshold)

In [12]:
def estimate_state_values(P, m2, threshold):
    v = np.zeros(m2 + 1)
    max_change = threshold
    terminal_state = m2
    while max_change >= threshold:
        max_change = 0
        for s in range(m2 + 1): # dla każdego stanu
            v_new = 0
            for s_next in range(m2 + 1): # aktualizacja wartości stanów
                r = 1 * (s_next != terminal_state)
                v_new += P[s, s_next] * (r + v[s_next])
            max_change = max(max_change, np.abs(v[s] - v_new))
            v[s] = v_new
    return np.round(v, 2)

In [13]:
estimate_state_values(P, m2, 0.005)

array([1.47, 2.12, 1.47, 2.44, 3.42, 2.44, 1.99, 2.82, 1.99, 0.  ])

### Zadanie #4
Dlaczego to podejście jest niepraktyczne w praktyce? Podaj dwa powody.

*Tip: ma to związek z przestrzenią stanów oraz macierzą P.*

1) duża złożoność obliczeniowa funkcji <br>
Mamy pętle while i dwie pętle for, przy dużym środowisku, algorytm może wykonywać się bardzo długo. <br>

2) generacja całej macierzy P (algorytm sprawdza przechodzenie z każdego stanu do każdego - nieoptymalne)

### Zadanie #5

Dokonaj estymacji wartości stanu dla środowisk o rozmiarach:
* 10x10
* 50x50
* 100x100
* 500x500
* 1000x1000

Zmierz czas obliczeń i przedstaw wyniki w tabeli. Opisz obserwacje.

Dla środowisk o rozmiarach: 100x100, 500x500, 1000x1000 program za długo się wykonuje, dlatego zrobiłam tylko dla pierwszych dwóch.

In [14]:
def get_matrix_P(m):
    m2 = m ** 2
    q = np.zeros(m2)
    q[m2 // 2] = 1
    P = np.zeros((m2 + 1, m2 + 1))
    P[:m2, :m2] = get_P(m, 0.2, 0.3, 0.25, 0.25)
    for i in range(m2):
        P[i, m2] = P[i, i]
        P[i, i] = 0
    P[m2, m2] = 1
    return P

In [ ]:
import time
l = [10, 50]
e = []
t = []
for m in l:
    P = get_matrix_P(m)
    start_time = time.time()
    e.append(estimate_state_values(P, m*m, 0.005))
    end_time = time.time()
    t.append(end_time - start_time)
print(e)
print(t)